In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True, src_lang="uig_Arab")
# tokenizer_target = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_auth_token=True, src_lang="eng_Latn")

# # prepare
# from datasets import load_dataset
# raw_eval_dataset = load_dataset("Muennighoff/flores200", "eng_Latn-uig_Arab",  cache_dir='/home/zzy/nfsdata/flores200')


# def tokenize_function(examples):
#     return tokenizer(examples["sentence_uig_Arab"], padding="max_length", truncation=True)

# def tokenize_function_target(examples: dict):
#     return tokenizer( examples["sentence_eng_Latn"], padding="max_length", truncation=True)


# tokenized_eval_dataset = raw_eval_dataset.map(tokenize_function, batched=True)['devtest']
# tokenized_eval_dataset_target = raw_eval_dataset.map(tokenize_function_target, batched=True)['devtest']
# labels = tokenized_eval_dataset_target['input_ids']
# tokenized_eval_dataset = tokenized_eval_dataset.add_column(name='labels', column=labels)

# tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink','sentence_uig_Arab'])

# tokenized_eval_dataset.set_format("torch")
# print(tokenized_eval_dataset.column_names)

# from torch.utils.data import DataLoader
# eval_dataloader = DataLoader(
#     tokenized_eval_dataset, batch_size=1, shuffle=False
# )

from datasets import load_from_disk, load_dataset

dataset_token_train_uig = load_from_disk('/home/zzy/nfsdata/nllb-token-uig')
print(dataset_token_train_uig['train'].column_names)
print(dataset_token_train_uig['train'].num_rows)

dataset_token_train_eng = load_from_disk('/home/zzy/nfsdata/nllb-token-eng')
print(dataset_token_train_eng['train'].column_names)
print(dataset_token_train_eng['train'].num_rows)

labels = dataset_token_train_eng['train']['input_ids']
dataset_token_train_uig['train'] = dataset_token_train_uig['train'].add_column(name='labels', column=labels)
dataset_token_train_uig.save_to_disk('/home/zzy/nfsdata/nllb-token')
# train_dataloader = DataLoader(
#     dataset_token_train_uig['train'] , batch_size=1, shuffle=False
#)


['input_ids', 'attention_mask']
9846570


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, T5Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
print(torch.cuda.is_available())
t5config = T5Config.from_pretrained('t5-small')
t5config.vocab_size = 256206
model = T5ForConditionalGeneration(t5config)
model.init_weights()
model.cuda()
model_teacher = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
model_teacher.cuda()


True


M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): Embedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): Embedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0): M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,)

In [3]:
for batch in eval_dataloader:
    break

batch.keys()
b = {'input_ids':batch['input_ids'], 'attention_mask': batch['attention_mask'], 'labels':batch['labels']}
output = model(batch['input_ids'].cuda(),batch['attention_mask'].cuda(), batch['labels'].cuda())
print(output.logits.shape)

torch.Size([1, 1024, 256206])


In [3]:
for batch in eval_dataloader:
    break
output = model_teacher(batch['input_ids'].cuda(),batch['attention_mask'].cuda(), batch['labels'].cuda())
print(output.logits.shape)

torch.Size([1, 1024, 256206])


In [4]:

from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=6e-5)
from transformers import get_scheduler
num_training_steps = len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

from tqdm.auto import tqdm
import torch.nn.functional as F
num_epochs = 20
num_training_steps = num_epochs * len(train_dataloader)
model.train()
model_teacher.eval()
leniter = len(train_dataloader)
for epoch in range(num_epochs):
    print('Epoch ', epoch, 'start')
    for i, batch in enumerate(train_dataloader):
        batch = {k: v.to('cuda') for k, v in batch.items()}
        # only need input
        outputs = model(batch['input_ids'],batch['attention_mask'], batch['labels'])
        #
        distill_out = model_teacher(batch['input_ids'],batch['attention_mask'], batch['labels']).detach() 
        
        loss = outputs.loss + 0.5 * F.mse_loss(outputs.logits, distill_out.logits)
        
        loss.backward()
        
        print('Epoch {:d}/{:d},  Iter {:d}/{:d} Loss: {:.4f}'.format(epoch + 1, num_epochs, i + 1, len(leniter), loss.item()))
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
    # save model per 5 epoch
    if epoch%5 == 0:
        torch.save({
            'epoch': epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": lr_scheduler.state_dict(),
        },
            "saved_model{:d}.pt".format(epoch))

import evaluate
metric = evaluate.load("chrf")
num_training_steps = len(eval_dataloader)
progress_bar = tqdm(range(num_training_steps))
for batch in eval_dataloader:
    label = batch["sentence_eng_Latn"]
    batch = {'input_ids': batch['input_ids'], 'attention_mask': batch['attention_mask']}
    batch = {k: v.to('cuda') for k, v in batch.items()}
    output = model.generate(**batch)
    predictions = tokenizer.batch_decode(output, skip_special_tokens=True)
    # print(tokenizer.batch_decode(batch['input_ids']),predictions,label)
    metric.add_batch(predictions=predictions, references=label)
    progress_bar.update(1)
metric.compute(word_order=2)



/home/zzy/miniconda3/envs/chatglm/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NameError: name 'train_dataloader' is not defined

In [6]:
import time
timestamp = int(time.time())
timestamp = time.strftime("%Y.%m%d.%H:%M:%S", time.localtime(timestamp))
print(timestamp)

2023.0720.21:35:41
